# **Fake News Detection - Method 2 (Training)**

Ignore warnings

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

Constants and hyperparameters

In [2]:
from pathlib import Path


# Constants
DATASETS = ["ISOT", "LIAR"]
PATHS = {
    "ISOT": {
        "train": "data/processed/ISOT/isot_train.pkl",
        "test": "data/processed/ISOT/isot_test.pkl",
        "aux": "data/features/ISOT/bow_min30_chi2700.joblib"
    },
    "LIAR": {
        "train": "data/processed/LIAR/liar_train.pkl",
        "test": "data/processed/LIAR/liar_test.pkl",
        "aux": "data/features/LIAR/tfidf_min40_chi2700.joblib"
    }
}
SAVED_MODELS_PATH = Path("saved_models/method2")

# Hyperparameters
W2V_DIMS = [100, 200] # Word2Vec dimensions
WINDOW_SIZES = [5, 8] # Word2Vec window sizes
MAX_LEN = 250 # pad length

Some functions to build model

In [3]:
import numpy as np
from gensim.models import Word2Vec


# Split text (into tokens)
def split_text(text):
    return text.split()

# Build Word2Vec
def build_w2v(sentences, dim, window):
    model = Word2Vec(
        sentences=sentences,
        vector_size=dim, # dimension
        window=window, # window size
        min_count=2, # minimum count
        sg=1, # skip-gram
        workers=4,
        epochs=5,
        seed=42
    )
    return model

# Build embedding matrix
def build_embedding_matrix(word_index, w2v_model, dim):
    # Initialize embedding matrix for all tokens with small random values
    emb = np.random.uniform(-0.05, 0.05, (len(word_index) + 1, dim)).astype(np.float32)

    # Update embedding matrix with Word2Vec vectors for known words
    for word, idx in word_index.items():
        if word in w2v_model.wv:
            emb[idx] = w2v_model.wv[word]

    return emb

Train model(Bi-LSTM)

In [4]:
from tensorflow.keras import mixed_precision # type: ignore
mixed_precision.set_global_policy("mixed_float16")

import itertools
import joblib
from scipy import sparse
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks, optimizers # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from keras.preprocessing.sequence import pad_sequences


for ds in DATASETS:
    # Load train set and features that were best at method 1
    train_df = joblib.load(PATHS[ds]["train"])
    aux_obj = joblib.load(PATHS[ds]["aux"]) # method 1 features
    X_aux = aux_obj["X"]
    if sparse.issparse(X_aux): X_aux = X_aux.toarray().astype("float16")
    y = train_df["label"].values
    
    # Split text that was already cleaned
    tokens_list = train_df["cleaned"].apply(split_text).tolist()
    
    for dim, win in itertools.product(W2V_DIMS, WINDOW_SIZES):
        tag = f"{ds}_{dim}d_win{win}"
        print(f"\nTraining on: dataset={ds}, w2v_dim={dim}, w2v_window_size={win} model=Bi-LSTM")
        
        # Build Word2Vec
        w2v_path = SAVED_MODELS_PATH / "w2v" / f"{tag}.model"
        if w2v_path.exists():
            w2v_model = Word2Vec.load(str(w2v_path))
        else:
            w2v_model = build_w2v(tokens_list, dim, win)
            w2v_model.save(str(w2v_path))
        
        # Build tokenizer and tokenize
        tok_path = SAVED_MODELS_PATH / "tokenizer" / f"{ds}.pkl"
        if tok_path.exists():
            tok = joblib.load(tok_path)
        else:
            tok = Tokenizer(oov_token="<UNK>", filters="") # when tokenizer encounters a new word, it is marked as unknown
            tok.fit_on_texts(tokens_list) # gives each word an id
            joblib.dump(tok, tok_path)
        
        # Create sequences
        sequences = tok.texts_to_sequences(tokens_list)
        X_seq = pad_sequences(sequences, maxlen=MAX_LEN, padding="post", truncating="post") # padding for fitting into input vector of Bi-LSTM
        
        # Build embedding matrix
        emb_matrix = build_embedding_matrix(tok.word_index, w2v_model, dim)
        
        # Split dataset into train and validation
        X1_tr, X1_val, X2_tr, X2_val, y_tr, y_val = train_test_split(X_seq, X_aux, y, test_size=0.2, stratify=y, random_state=42)
        
        # Create input layers
        seq_input = layers.Input(shape=(MAX_LEN,), name="seq_input") # sequences
        aux_input = layers.Input(shape=(X_aux.shape[1],), name="aux_input") # method 1 features
        
        # Embedding layer
        emb = layers.Embedding(
            input_dim=emb_matrix.shape[0],
            output_dim=dim,
            weights=[emb_matrix],
            input_length=MAX_LEN,
            trainable=False,
            name="embedding",
        )(seq_input)
        
        # Bi-LSTM layers
        x = layers.SpatialDropout1D(0.25)(emb)

        x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
        x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
        x = layers.GlobalMaxPooling1D()(x)
        
        merged = layers.concatenate([x, aux_input]) # merge LSTM output vector with method 1 features
        merged = layers.Dropout(0.4)(merged) # randomly turns neurons off to prevent overfitting
        merged = layers.Dense(128, activation="relu")(merged)
        out = layers.Dense(1, activation="sigmoid", dtype="float32")(merged)
        
        # Build model
        model = models.Model([seq_input, aux_input], out, name=f"BiLSTM_{tag}")
        model.compile(
            optimizer=optimizers.Adam(learning_rate=1e-3),
            loss="binary_crossentropy",
            metrics=["accuracy"]
        )
        model.summary(line_length=120)
        
        # Callbacks
        ckpt_path = SAVED_MODELS_PATH / "bilstm" / f"{tag}.keras"
        callback_list = [
            callbacks.ModelCheckpoint(ckpt_path, monitor="val_loss", save_best_only=True, verbose=1), # saves only best model
            callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1), # reduces learning rate adaptively
            callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1) # early stopping after 5 unimproved epochs
        ]
        
        # Training model
        model.fit(
            x=[X1_tr, X2_tr], y=y_tr,
            validation_data=([X1_val, X2_val], y_val),
            epochs=15,
            batch_size=512,
            verbose="auto",
            callbacks=callback_list
        )

2025-07-25 02:17:40.517688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753399060.529201  163962 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753399060.532606  163962 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753399060.542757  163962 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753399060.542772  163962 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753399060.542773  163962 computation_placer.cc:177] computation placer alr


Training on: dataset=ISOT, w2v_dim=100, w2v_window_size=5 model=Bi-LSTM


I0000 00:00:1753399069.302677  163962 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "BiLSTM_ISOT_100d_win5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Layer (type)                      ┃ Output Shape                 ┃           Param # ┃ Connected to              
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ seq_input (InputLayer)            │ (None, 250)                  │                 0 │ -                         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ embedding (Embedding)             │ (None, 250, 100)             │        16,755,900 │ seq_input[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ spatial_dropout1d                 │ (None, 250, 100)             │                 0 │ embedding[0][0]           
│ (SpatialDropout1D)                │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ bidirectional (Bidirectional)     │ (None, 250, 256)             │           234,496 │ spatial_dropout1d[0][0]   
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ bidirectional_1 (Bidirectional)   │ (None, 250, 128)             │           164,352 │ bidirectional[0][0]       
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ global_max_pooling1d              │ (None, 128)                  │                 0 │ bidirectional_1[0][0]     
│ (GlobalMaxPooling1D)              │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ aux_input (InputLayer)            │ (None, 700)                  │                 0 │ -                         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ concatenate (Concatenate)         │ (None, 828)                  │                 0 │ global_max_pooling1d[0][0]
│                                   │                              │                   │ aux_input[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dropout (Dropout)                 │ (None, 828)                  │                 0 │ concatenate[0][0]         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dense (Dense)                     │ (None, 128)                  │           106,112 │ dropout[0][0]             
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dense_1 (Dense)                   │ (None, 1)                    │               129 │ dense[0][0]               
└───────────────────────────────────┴──────────────────────────────┴───────────────────┴───────────────────────────

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Layer (type)                      ┃ Output Shape                 ┃           Param # ┃ Connected to              
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ seq_input (InputLayer)            │ (None, 250)                  │                 0 │ -                         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ embedding (Embedding)             │ (None, 250, 200)             │         1,915,800 │ seq_input[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ spatial_dropout1d_7               │ (None, 250, 200)             │                 0 │ embedding[0][0]           
│ (SpatialDropout1D)                │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ bidirectional_14 (Bidirectional)  │ (None, 250, 256)             │           336,896 │ spatial_dropout1d_7[0][0] 
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ bidirectional_15 (Bidirectional)  │ (None, 250, 128)             │           164,352 │ bidirectional_14[0][0]    
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ global_max_pooling1d_7            │ (None, 128)                  │                 0 │ bidirectional_15[0][0]    
│ (GlobalMaxPooling1D)              │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ aux_input (InputLayer)            │ (None, 561)                  │                 0 │ -                         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ concatenate_7 (Concatenate)       │ (None, 689)                  │                 0 │ global_max_pooling1d_7[0][
│                                   │                              │                   │ aux_input[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dropout_7 (Dropout)               │ (None, 689)                  │                 0 │ concatenate_7[0][0]       
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dense_14 (Dense)                  │ (None, 128)                  │            88,320 │ dropout_7[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dense_15 (Dense)                  │ (None, 1)                    │               129 │ dense_14[0][0]            
└───────────────────────────────────┴──────────────────────────────┴───────────────────┴───────────────────────────

 Total params: 2,505,497 (9.56 MB)

 Trainable params: 589,697 (2.25 MB)

 Non-trainable params: 1,915,800 (7.31 MB)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.5336 - loss: 0.6858
Epoch 1: val_loss improved from inf to 0.67122, saving model to saved_models/method2/bilstm/LIAR_200d_win8.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 161ms/step - accuracy: 0.5350 - loss: 0.6854 - val_accuracy: 0.5828 - val_loss: 0.6712 - learning_rate: 0.0010
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.5767 - loss: 0.6701
Epoch 2: val_loss improved from 0.67122 to 0.65952, saving model to saved_models/method2/bilstm/LIAR_200d_win8.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 138ms/step - accuracy: 0.5771 - loss: 0.6699 - val_accuracy: 0.5950 - val_loss: 0.6595 - learning_rate: 0.0010
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.6150 - loss: 0.6583
Epoch 3: val_loss improved from 0.65952 to 0.65861, saving model to saved_models/method2/bilstm/LIAR_200d_win8.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 137ms/step - accuracy: 0.6147 - loss: 0.6584 - val_accuracy: 0.6019 - val_loss: 0.6